## Import Autophagy Package

In [ ]:
# Specific imports for cellular_raza
# The package is named cr_autophagy
# We want to reload the package when some of the behind-the scenes python functions change
# This is what the importlib statements are for
import importlib
import cr_autophagy as cra
importlib.reload(cra)

# Imports of general-purpose python libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pyvista as pv
import multiprocessing as mp
from pathlib import Path
import os

output_path = cra.get_last_output_path()
simulation_settings = cra.get_simulation_settings(output_path)
iter_0_particles = cra.get_particles_at_iter(output_path, 0)
for col in iter_0_particles.columns:
    print(col)
max_iter = max(cra.get_all_iterations(output_path))

## Read results from json Files
The results of the simulation are saved in json files.
Due to the parallelized nature of the simulation, not all results are in one big json file but rather in multiple batches. We therefore need to combine these batches to obtain a complete set for a given iteration.

We want to inspect which entries our generated dataset has. Therefore, we normalize the dict, transforming it into a dataframe.
Afterwards, we display all columns.

## Analysis
1. Determine how the Cargo size changes over the course of the simulation
2. Determine if ATG11Receptor actually clusters

In [ ]:
def save_scatter_snapshot(iteration):
    df = cra.get_particles_at_iter(output_path, iteration)

    cargo_at_end = df[df["element.cell.interaction.species"]=="Cargo"]["element.cell.mechanics.mechanics.pos"]
    cargo_at_end = np.array([np.array(elem) for elem in cargo_at_end])
    non_cargo_at_end = df[df["element.cell.interaction.species"]!="Cargo"]["element.cell.mechanics.mechanics.pos"]
    non_cargo_at_end = np.array([np.array(elem) for elem in non_cargo_at_end])
    cargo_middle = np.average(non_cargo_at_end, axis=0)

    def appendSpherical_np(xyz):
        ptsnew = np.hstack((xyz, np.zeros(xyz.shape)))
        xy = xyz[:,0]**2 + xyz[:,1]**2
        ptsnew[:,3] = np.sqrt(xy + xyz[:,2]**2)
        ptsnew[:,4] = np.arctan2(np.sqrt(xy), xyz[:,2]) # for elevation angle defined from Z-axis down
        #ptsnew[:,4] = np.arctan2(xyz[:,2], np.sqrt(xy)) # for elevation angle defined from XY-plane up
        ptsnew[:,5] = np.arctan2(xyz[:,1], xyz[:,0])
        return ptsnew

    non_cargo_at_end_spherical = appendSpherical_np(non_cargo_at_end - cargo_middle)
    r = non_cargo_at_end_spherical[:,3]
    r_inv = np.max(r) - r
    phi = non_cargo_at_end_spherical[:,4]
    theta = non_cargo_at_end_spherical[:,5]

    fig, ax = plt.subplots()
    ax.set_title("Radial distribution of particles around cargo center")
    ax.scatter(phi, theta, s=r_inv, alpha=0.5)

    ax.set_xlabel("$\\varphi$ [rad]")
    ax.set_ylabel("$\\theta$ [rad]")
    ax.set_xticks([0, np.pi/4, np.pi/2, 3*np.pi/4, np.pi])
    ax.set_xticklabels(["$0$", "$\\frac{\\pi}{4}$", "$\\frac{\\pi}{2}$", "$\\frac{3\\pi}{4}$", "$\\pi$"])
    ax.set_yticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
    ax.set_yticklabels(["$-\\pi$", "$-\\frac{\\pi}{2}$", "$0$", "$\\frac{\\pi}{2}$", "$\\pi$"])

    ax.set_xlim([-np.pi/12, np.pi*(1+1/12)])
    ax.set_ylim([-np.pi*(1+1/6), np.pi*(1+1/6)])

    ofolder = output_path / "scatterplots"
    ofolder.mkdir(parents=True, exist_ok=True)
    fig.savefig(ofolder / f"snapshot_{iteration:08}_scatter.png")
    plt.close(fig)

In [ ]:
with mp.Pool() as p:
    p.map(save_scatter_snapshot, cra.get_all_iterations(output_path))

In [ ]:
# Also create a movie with ffmpeg
bashcmd = f"ffmpeg -y -r 30 -f image2 -pattern_type glob -i '{output_path}/scatterplots/*.png' -c:v h264 -pix_fmt yuv420p -strict -2 {output_path}/scatter_movie.mp4"
os.system(bashcmd)

## Plot Result
We visualize the results in 3D.
Therefore we use `pyvista` which internally uses `vtk` as a backend.
Since all of our particles are represented as 3D-spheres, we also display them as such.

In [ ]:
def generate_spheres(iteration):
    # Filter for only particles at the specified iteration
    df = cra.get_particles_at_iter(output_path, iteration)
    # df = df[df["iteration"]==iteration]

    # Create a dataset for pyvista for plotting
    pset = pv.PolyData(np.array([np.array(x) for x in df["element.cell.mechanics.mechanics.pos"]]))

    # Extend dataset by species and diameter
    pset.point_data["diameter"] = 2.0*df["element.cell.interaction.cell_radius"]
    pset.point_data["species"] = df["element.cell.interaction.species"]

    # Create spheres glyphs from dataset
    sphere = pv.Sphere()
    spheres = pset.glyph(geom=sphere, scale="diameter", orient=False)

    return spheres

def save_snapshot(iteration):
    ofolder = Path(output_path) / "snapshots"
    ofolder.mkdir(parents=True, exist_ok=True)
    opath = ofolder / "snapshot_{:08}.png".format(iteration)
    if os.path.isfile(opath):
        return
    spheres = generate_spheres(iteration)

    spheres.plot(
        off_screen=True,
        screenshot=opath,
        scalars="species",
        scalar_bar_args={
            "title":"Species",
        },
        cpos=[
            (
                -1.5*simulation_settings.domain_size,
                -1.5*simulation_settings.domain_size,
                -1.5*simulation_settings.domain_size
            ),(
                25,
                25,
                25
            ),(
                0.0,
                0.0,
                0.0
            )
        ],
        jupyter_backend='none',
    )

We can save single snapshots or even use all processes of our device to save snapshots for every iteration.
The 2nd approach will take up all resources by default. If you want to limit this, have a look at the [Pool object of the multiprocessing](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.Pool) module.

In [ ]:
# Save all snapshots
with mp.Pool() as p:
    p.map(save_snapshot, cra.get_all_iterations(output_path))

In [ ]:
# Also create a movie with ffmpeg
bashcmd = f"ffmpeg -y -r 30 -f image2 -pattern_type glob -i '{output_path}/snapshots/*.png' -c:v h264 -pix_fmt yuv420p -strict -2 {output_path}/snapshot_movie.mp4"
os.system(bashcmd)